<a href="https://colab.research.google.com/github/Praise-Atadja/EIICD_chatbox/blob/main/EIICD_chatbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from transformers import BertTokenizer
import os
import numpy as np

In [ ]:

def question_answer(question, reference):
    tok = BertTokenizer.from_pretrained(
        "bert-large-uncased-whole-word-masking-finetuned-squad")
    model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")

    quest_toks = tok.tokenize(question)
    text_toks = tok.tokenize(reference)
    tokens = ['[CLS]'] + quest_toks + [
        '[SEP]'] + text_toks + ['[SEP]']

    input_id = tok.convert_tokens_to_ids(tokens)
    mask = [1] * len(input_id)
    input_type = [0] * (1 + len(quest_toks) + 1) + [1] * (len(text_toks) + 1)

    input_id, mask, input_type = map(lambda x: tf.expand_dims(
        tf.convert_to_tensor(x, tf.int32), 0), (input_id, mask, input_type))

    output = model([input_id, mask, input_type])
    start = tf.argmax(output[0][0][1:]) + 1
    end = tf.argmax(output[1][0][1:]) + 1
    answer_tok = tokens[start: end + 1]
    answer = tok.convert_tokens_to_string(answer_tok)

    return answer

In [ ]:
"""Create the loop"""


while True:
    question = input("Q: ")
    texts = ["exit", "quit", "goodbye", "bye"]

    if question.lower().strip() in texts:
        print("A: Goodybe")
        exit(0)
    else:
        print("A: ")

In [ ]:
"""Answer some questions"""

def question_answer(question, reference):
    """Questions answer method"""
    tok = BertTokenizer.from_pretrained(
        "bert-large-uncased-whole-word-masking-finetuned-squad")
    model = hub.load("https://tfhub.dev/see--/bert-uncased-tf2-qa/1")

    quest_toks = tok.tokenize(question)
    text_toks = tok.tokenize(reference)
    tokens = ['[CLS]'] + quest_toks + [
        '[SEP]'] + text_toks + ['[SEP]']

    input_id = tok.convert_tokens_to_ids(tokens)
    mask = [1] * len(input_id)
    input_type = [0] * (1 + len(quest_toks) + 1) + [1] * (len(text_toks) + 1)

    input_id, mask, input_type = map(lambda x: tf.expand_dims(
        tf.convert_to_tensor(x, tf.int32), 0), (input_id, mask, input_type))

    output = model([input_id, mask, input_type])
    start = tf.argmax(output[0][0][1:]) + 1
    end = tf.argmax(output[1][0][1:]) + 1
    answer_tok = tokens[start: end + 1]
    answer = tok.convert_tokens_to_string(answer_tok)

    return answer


def answer_loop(reference):
    """Answer question from reference"""
    while True:
        question = input("Q: ")
        question = question.lower().strip()
        leaving = ["exit", "quit", "goodbye", "bye"]

        if question in leaving:
            print("A: Goodbye")
            break

        answer = question_answer(question, reference)
        if answer is None:
            print("A: Sorry, I do not understand your question.")
        else:
            print("A: ", answer)


In [ ]:
"""Semantic Search task"""

def semantic_search(corpus_path, sentence):
    """Perform semantic search on corpus of documents"""
    doc = [sentence]
    model = hub.load('https://tfhub.dev/google/universal-sentence' +
                     '-encoder-large/5')

    for filename in os.listdir(corpus_path):
        if not filename.endswith('.md'):
            continue
        with open(corpus_path + '/' + filename, 'r', encoding='utf-8') as f:
            doc.append(f.read())

    embs = model(doc)
    comp = np.inner(embs, embs)
    close = np.argmax(comp[0, 1:])
    alike = doc[close + 1]

    return alike